# Аналитический кейс: анализ клиентской активности и покупок

Цель проекта — проанализировать поведение клиентов сервиса, их активность и покупательские паттерны,
а также выявить ключевые группы пользователей и особенности их взаимодействия с продуктом.

Анализ выполнен с использованием Python и библиотек pandas, numpy, seaborn.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Данные и предварительная обработка

В анализе используются обезличенные данные о клиентах, их активности и покупках.
На этом этапе выполняется загрузка данных, первичная очистка и подготовка таблиц для дальнейшего анализа.

In [ ]:
users_info = pd.read_csv('users_info.csv')
users_info.head()

## Анализ активности клиентов

В этом разделе анализируется активность клиентов: количество действий,
частота взаимодействий и распределение пользователей по уровню вовлеченности.
Хотим понять, как различается поведение активных и менее активных пользователей.

In [ ]:
users_info.shape

In [ ]:
users_info.dtypes

In [ ]:
len(users_info.loc[users_info['id_user'].isnull()])

In [ ]:
len(users_info.loc[users_info['city'].isnull()])

In [ ]:
len(users_info.loc[users_info['age'].isnull()])

In [ ]:
len(users_info.loc[users_info['gender'].isnull()])

In [ ]:
users_info = users_info[~users_info['age'].isnull()] 

In [ ]:
users_info = users_info[~users_info['gender'].isnull()] 
users_info

In [ ]:
1 - len(users_info) /1000 # удалили немного 13.5% данных 

In [ ]:
Q1 = np.percentile(users_info['age'], 25)
Q3 = np.percentile(users_info['age'], 75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(lower_bound)
print(upper_bound)

In [ ]:
users_clean = users_info.loc[users_info['age'] < upper_bound].copy()
users_clean.head(10)

In [ ]:
sns.histplot(users_clean['age'], kde = True)

## Анализ покупательского поведения

Рассматриваются данные о покупках клиентов, включая количество транзакций
и распределение расходов.
Анализ помогает оценить вклад разных групп пользователей в общую выручку.

In [ ]:
users_cnt = users_clean.groupby(['city', 'gender']).agg(cnt_users = ('id_user', 'count')).reset_index()
users_cnt

In [ ]:
pivot_gender_city = users_clean.pivot_table(
    index = 'city',
    columns = 'gender',
    values = 'id_user',
    aggfunc = 'count')
pivot_gender_city

In [ ]:
pivot_gender_city['fem_share'] = pivot_gender_city['female'] / (pivot_gender_city['female'] + pivot_gender_city['male'])
pivot_gender_city                                                                

In [ ]:
pivot_gender_city['fem_share'].plot(kind = 'bar') # Доля женщин в Москве сильно выше, чем в других городах, около 70%
plt.ylabel('Доля женщин')
plt.xlabel('Город')
plt.title('Доля женщин по городам')
plt.grid(axis = 'y')
plt.show()

## Сегментация клиентов

Клиенты разделены на группы на основе активности и покупательских характеристик.
Сравниваются основные показатели сегментов и их поведенческие различия.

In [ ]:
pay_mon = pd.read_csv('payments_monthly.csv')
pay_mon.head()

In [ ]:
user_activ = pay_mon.groupby(pay_mon['id_user']).agg(avg_cnt = ('cnt_total', 'mean')).reset_index()
user_activ.head()                          

In [ ]:
user_activ['activ_group'] = pd.cut(user_activ['avg_cnt'],
                                   bins = [0, 3, 6, user_activ['avg_cnt'].max()],
                                   labels = ['мало', 'средне', 'много'], include_lowest = True)
user_activ.head()

In [ ]:
activ_stats = user_activ.groupby('activ_group', observed = False).agg(cnt_activ = ('id_user', 'count')).reset_index()
activ_stats

## Выводы

- Поведение клиентов существенно различается в зависимости от уровня активности.
- Небольшая доля пользователей формирует значительную часть покупок.
- Сегментация позволяет выделить группы с наибольшей ценностью для продукта.
- Полученные результаты могут быть использованы для дальнейшего анализа и принятия продуктовых решений.

Кейс демонстрирует применение Python для анализа клиентских данных и выявления поведенческих паттернов.